In [2]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras_self_attention import SeqSelfAttention
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



In [3]:

snli_train=Table.read_table("dataset/SNLI/snli_1.0_train.txt") 

snli_dev=Table.read_table("dataset/SNLI/snli_1.0_dev.txt") 

snli_test=Table.read_table("dataset/SNLI/snli_1.0_test.txt") 


/usr/local/lib/python3.5/dist-packages/datascience/tables.py:132: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  df = pandas.read_table(filepath_or_buffer, *args, **vargs)


In [4]:
dataframe = pd.read_csv("dataset/ppdb/sanitized.csv")
dataframe.to_csv("dataset/ppdb/sanitized.tsv", sep='\t', index=False)

In [5]:
ppdb_Tb=Table.read_table("dataset/ppdb/sanitized.tsv")

In [6]:
train_id = snli_train.column("pairID")
train_sen1=snli_train.column("sentence1")
train_sen2=snli_train.column("sentence2")  #SNLI
train_gold = snli_train.column("gold_label")


dev_id = snli_dev.column("pairID")
dev_sen1=snli_dev.column("sentence1")
dev_sen2=snli_dev.column("sentence2")  #SNLI
dev_gold = snli_dev.column("gold_label")

test_id = snli_test.column("pairID")
test_sen1=snli_test.column("sentence1")
test_sen2=snli_test.column("sentence2")  #SNLI
test_gold = snli_test.column("gold_label")

phrase = ppdb_Tb.column("PHRASE")
paraphrase=ppdb_Tb.column("PARAPHRASE")   #ppdb


In [31]:
test_id

array(['2677109430.jpg#1r1n', '2677109430.jpg#1r1e',
       '2677109430.jpg#1r1c', ..., '152881593.jpg#1r1c',
       '152881593.jpg#1r1e', '152881593.jpg#1r1n'], dtype='<U19')

In [ ]:
index_list=[]          #test
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
gold = []

ans = []
for x in range(len(test_id)):   #index of SNLI
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in test_sen1[x] and paraphrase[z] in test_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, test_sen1[x])
            sent2=np.append(sent2, test_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            gold = np.append(gold, test_gold[x])

 seems to be 
 seems to be 
 seems to be 
 seems to 
 seems to be 
 seems to be 
 seems to 
 seems to be 
 seems to 
 seems to be 
 seems to 
 standing in 
 playing in 
 playing in 
 younger people 
 younger people 
 younger people 
 young man 
 stands in 
 is about to 
 is about to 
 another man 
 another man 
 another man 
 another person 
 young man 
 young man 
 sitting down 
 getting ready 
 another man 
 another man 
 another man 
 playing around 
 playing around 
 plays in 
 plays in 
 play in 
 play in 
 another person 
 person who is 
 somebody else 
 speaks to 
 young man 
 getting ready to 
 of men 
 young men 
 young men 
 young men 
 young man 
 young man 
 talk to each other 
 plays in 
 plays in 
 little girl 
 little girl 
 little girl 
 a photograph 
 get to work 
 is sitting 
 construction work 
 construction work 
 walk away 
 walk away 
 sitting down 
 standing in 
 very large 
 waiting outside 
 waiting outside 
 young men 
 young men 
 young men 
 young men 
 high

In [ ]:
index_list=[]          #dev
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
gold = []

ans = []
for x in range(len(dev_id)):   #index of SNLI
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in dev_sen1[x] and paraphrase[z] in dev_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, dev_id[x])
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, dev_sen1[x])
            sent2=np.append(sent2, dev_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            gold = np.append(gold, dev_gold[x])

 young man 
 middle east 
 middle east 
 middle east 
 sitting outside 
 another man 
 little girl 
 and women 
 little girl 
 little girl 
 little girl 
 young man 
 young man 
 young man 
 sit around 
 sit around 
 sit around 
 stands in 
 little girl 
 little girl 
 another man 
 another man 
 standing in 
 standing in 
 young man 
 young man 
 young man 
 running around 
 stands at 
 sitting in 
 walking around 
 young man 
 young man 
 young man 
 two children 
 two children 
 two children 
 little girl 
 stand in 
 works on 
 works on 
 another man 
 another man 
 another man 
 another man 
 another man 
 young men 
 young men 
 another man 
 another man 
 another man 
 young man 
 young man 
 young man 
 little girl 
 little girl 
 young people 
 little girl 
 little girl 
 little girl 
 little girl 
 little girl 
 little girl 
 little girl 
 young man 
 little girl 
 little girl 
 young man 
 young man 
 young man 
 stands behind 
 standing in 
 run around 
 reading a 
 is work

In [ ]:
phrase_dataset = Table().with_column("SNLI_PairID", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "gold_label", gold)

In [10]:
phrase_dataset.show(4)

SNLI_PairID,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,gold_label
10,20265,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment
10,34976,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment
10,61957,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment
10,125335,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to,seem to,entailment


In [37]:
phrase_dataset=phrase_dataset.with_columns("SNLI_PairID", hue)
phrase_dataset.show(4)

SNLI_PairID,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,gold_label,labels1,labels2
6526219567.jpg#4r1e,20265,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment,O O O O O O O B I E O O,O O O O O O O O B I E O O
6526219567.jpg#4r1e,34976,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment,O O O O O O O B I E O O,O O O O O O O O B I E O O
6526219567.jpg#4r1e,61957,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment,O O O O O O O B I E O O,O O O O O O O O B I E O O
6526219567.jpg#4r1e,125335,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to,seem to,entailment,O O O O O O O B E O O O,O O O O O O O O B E O O O


In [39]:
phrase_dataset=phrase_dataset.to_df()
phrase_dataset.to_csv("dataset/Phrase_Level/SNLI/PPDB_SNLI_TEST.tsv",sep='\t')

In [ ]:
index_list=[]
PPDB_id = []
sent1=[]
sent2=[]
para_list1=[]
para_list2=[]
labeling = []

ans = []
for x in range(len(dev_id)):   #index of PAWS_QQP
    
    for z in range(len(phrase)):  #index of PPDB
        if (phrase[z] in dev_sen1[x] and paraphrase[z] in dev_sen2[x]):
            print(phrase[z])
            index_list=np.append(index_list, x)
            PPDB_id=np.append(PPDB_id, z)
            sent1=np.append(sent1, dev_sen1[x])
            sent2=np.append(sent2, dev_sen2[x])
            para_list1=np.append(para_list1, phrase[z])
            para_list2=np.append(para_list2, paraphrase[z])
            labeling = np.append(labeling, dev_label[x])

In [ ]:
phrase_dataset = Table().with_column("DEV_PAWS_QQP_id", index_list, "PPDB_id", PPDB_id, "sent1", sent1, "sent2", sent2, "paraphrase1", para_list1, "paraphrase2", para_list2, "label", labeling)

In [ ]:
phrase_dataset.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.csv")

In [71]:
dataframe = pd.read_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_TRAIN-Copy1.csv")

In [72]:
dataframe.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_TRAIN.tsv", sep='\t', index=False)

In [73]:
dataframe = pd.read_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV-Copy1.csv")
dataframe.to_csv("dataset/Phrase_Level/PPDB_PAWS_QQP_DEV.tsv", sep='\t', index=False)

In [32]:
phrase_dataset=Table.read_table('dataset/Phrase_Level/SNLI/SNLI_PPDB_TEST.tsv')

In [33]:
phrase_dataset.show(4)

SNLI_PairID,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,gold_label,labels1,labels2
10,20265,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment,O O O O O O O B I E O O,O O O O O O O O B I E O O
10,34976,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment,O O O O O O O B I E O O,O O O O O O O O B I E O O
10,61957,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to be,seem to be,entailment,O O O O O O O B I E O O,O O O O O O O O B I E O O
10,125335,A statue at a museum that no seems to be looking at.,There is a statue that not many people seem to be intere ...,seems to,seem to,entailment,O O O O O O O B E O O O,O O O O O O O O B E O O O


In [11]:

sent1=phrase_dataset.column("sent1")
sent2=phrase_dataset.column("sent2")
para1=phrase_dataset.column("paraphrase1")
para2=phrase_dataset.column("paraphrase2")

gg = [x.split() for x in sent1]
hh = [x.split() for x in sent2]
ii = [x.split() for x in para1]
jj = [x.split() for x in para2]



22

In [14]:
labels1=[]
for x in range(len(gg)):
    labels1.append(spit_label(gg[x], ii[x], len(ii[x]),len(ii[x]), []))



In [15]:
labels2=[]
for x in range(len(hh)):
    labels2.append(spit_label(hh[x], jj[x], len(jj[x]),len(jj[x]), []))



In [16]:
joined1=[]
for x in labels1:
    joined1.append(" ".join(x))

In [17]:
joined2=[]
for x in labels2:
    joined2.append(" ".join(x))

array(['2677109430.jpg#1r1n', '2677109430.jpg#1r1e',
       '2677109430.jpg#1r1c', ..., '152881593.jpg#1r1c',
       '152881593.jpg#1r1e', '152881593.jpg#1r1n'], dtype='<U19')

In [24]:
#phrase_dataset=phrase_dataset.with_columns("labels1", joined1, "labels2", joined2)
#phrase_dataset.show(4)
phrase_dataset=phrase_dataset.to_df()
phrase_dataset.to_csv("dataset/Phrase_Level/SNLI/SNLI_PPDB_TEST.tsv", sep='\t', index=False)

In [34]:
new_Tb

In [13]:
    def spit_label(split_sent, para_split, ori_length, length, listy):
       
    
        if para_split[0]==split_sent[0]:
            
            if ori_length==1:
                listy.append("S")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy    
            
            elif ori_length==length:
                listy.append("B")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
            elif length==1:
                listy.append("E")
                for x in range(len(split_sent)-1):
                    listy.append("O")
                return listy
            
            else:
                listy.append("I")
                return spit_label(split_sent[1:], para_split[1:], ori_length, length-1, listy)
                
        else:
            listy.append("O")
            return spit_label(split_sent[1:], para_split, ori_length, length, listy)

In [88]:
bbb="".join("he ")

In [89]:
bbb

'he '